In [1]:
# Descargamos el dataset
# !gdown --id 1u1MRRWnjoi3p5eWV6bOtJX4_bZ0DT4eu
# !unzip balanced_dataset.zip

In [2]:
# Librerias utilizadas.
import pandas as pd
import numpy as np
import tensorflow as tf
from skimage import io
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import Sequence
from tensorflow.keras.utils import to_categorical
import random
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Input,MaxPooling2D, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [3]:

# Generador de datos
class DataGenerator(Sequence):

  # imgIDs: Lista de las ID de las imagenes a utilizar en este generator.
  # labels: Diccionario de los labels para cada ID (sin one hot encoding)
  # batch_size: Tamaño de cada batch que se genera por llamado.
  # dim: Dimensiones de las imagenes.
  # n_channels: Numero de canales de la imagen.
  # n_classes: Numero de labels distintos.
  # shuffle: Si revolver los datos o no.
  def __init__(self, imgIDs, labels, batch_size=32, dim=(180, 320), n_channels=3, n_classes=4, shuffle=True):
    self.imgIDs = imgIDs
    self.labels = labels
    self.batch_size = batch_size
    self.dim = dim
    self.n_channels = n_channels
    self.n_classes = n_classes
    self.shuffle = shuffle
    self.on_epoch_end()

  # Retorna el numero de batches del generator.
  def __len__(self):
    return int(np.floor(len(self.imgIDs) / self.batch_size))

  # Retorna un batch.
  def __getitem__(self, index):
    indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
    list_IDs_temp = [self.imgIDs[k] for k in indexes]
    X, y = self.__data_generation(list_IDs_temp)

    return X, y

  # Cuando se termina un epoch, se escogen los siguientes ID.
  def on_epoch_end(self):
    self.indexes = np.arange(len(self.imgIDs))
    if self.shuffle == True:
      np.random.shuffle(self.indexes)

  # Se cargan los datos a medida que se necesitan.
  def __data_generation(self, list_IDs_temp):
    X = np.empty((self.batch_size, *self.dim, self.n_channels))
    y = np.empty((self.batch_size), dtype=int)

    for i, ID in enumerate(list_IDs_temp):
      X[i,] = io.imread("./balanced_dataset_medium/data/" + ID + ".jpg")
      y[i] = self.labels[ID]
    
    X = X.astype("float32") / 255

    return X, to_categorical(y, num_classes=self.n_classes)

In [4]:
# Observamos los datos del CSV.
dataInfo = pd.read_csv("./balanced_dataset_medium/data_info.csv")
dataInfo

,Unnamed: 0,ID,part,label,views,comments,likes,dislikes,topicID
0,0,id---2pGwkL7M_video,part1,0,3543,11.0,63.0,0.0,"['/m/04rlf', '/m/04rlf']"
1,1,id---9beuLW7Y_video,part1,0,1543,0.0,0.0,0.0,"['/m/098wr', '/m/098wr']"
2,2,id---dQKphvYc_video,part1,0,4065,4.0,13.0,0.0,"['/m/04rlf', '/m/064t9']"
3,3,id---EnHBffak_video,part1,0,7808,5.0,36.0,5.0,"['/m/0bzvm2', '/m/0bzvm2']"
4,4,id---FI9JLZRg_video,part1,0,1308,2.0,3.0,4.0,"['/m/0bzvm2', '/m/0bzvm2', '/m/0403l3g']"
...,...,...,...,...,...,...,...,...,...
19980,19980,idE2gK1mLbJKE_video,part189,3,3547841,460.0,16357.0,777.0,"['/m/04rlf', '/m/01lyv']"
19981,19981,ide2hvBdWy0X8_video,part189,3,1414097,568.0,8945.0,385.0,"['/m/04rlf', '/m/064t9', '/m/0glt670']"
19982,19982,ide2jPLqvMRwM_video,part189,3,1884647,112.0,2071.0,853.0,['/m/04rlf']
19983,19983,ide2JR9LtAic0_video,part189,3,1459441,3169.0,20779.0,386.0,"['/m/025zzc', '/m/0bzvm2', '/m/0bzvm2', '/m/02..."


In [5]:
# Tamaño de los grupos, donde se indica con proporciones.
# El resto se le asigna al grupo de test.
trainSize = 0.6
valSize = 0.2

# Generamos el diccionario de los labels, y cargamos todos los IDs en una lista.
labels = {}
data = []

for i, row in dataInfo.iterrows():
  data.append(row["ID"])
  labels[row["ID"]] = row["label"]

# Revolvemos la lista de ID.
random.Random(42).shuffle(data)

# Generamos las listas de IDs para cada grupo.
trainData = data[:int(len(data)*trainSize)]
valData = data[int(len(data)*trainSize) : int(len(data)*trainSize) + int(len(data)*valSize)]
testData = data[int(len(data)*trainSize) + int(len(data)*valSize):]

# Creamos el diccionario de listas de ID para cada grupo.
partition = {}
partition["train"] = trainData
partition["validation"] = valData
partition["test"] = testData

In [6]:
# TODO: Graficar una wea

In [7]:
# Creamos los generadores.
trainGenerator = DataGenerator(partition["train"], labels)
valGenerator = DataGenerator(partition["validation"], labels)
testGenerator = DataGenerator(partition["test"], labels)

In [17]:
# # Creamos la red y la compilamos.
# model = Sequential()
# #Conv 1
# model.add(Conv2D(96, kernel_size=(11, 11), activation='relu', input_shape=(180,320,3)))
# model.add(MaxPooling2D(pool_size=(3, 3)))

# #Conv 2
# model.add(Conv2D(256, kernel_size=(5, 5), activation="relu"))
# model.add(MaxPooling2D(pool_size=(3, 3)))

# #Conv 3
# model.add(Conv2D(384, kernel_size=(3, 3), activation="relu"))

# #Conv 4
# model.add(Conv2D(384, kernel_size=(3, 3), activation="relu"))

# #Conv 5
# model.add(Conv2D(256, kernel_size=(3, 3), activation="relu"))
# model.add(Dropout(0.50))

# model.add(Flatten())

# #FC 1
# model.add(Dense(4096, activation="relu"))
# model.add(Dropout(0.50))

# #FC 2
# model.add(Dense(4096, activation="relu"))
# model.add(Dropout(0.50))

# model.add(Dense(4, activation="softmax"))

model = Sequential()
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(180,320,3)))
model.add(Dropout(0.20))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation="relu"))
model.add(Dropout(0.20))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation="relu"))
model.add(Dropout(0.20))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation="relu"))
model.add(Dropout(0.20))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.50))
model.add(Dense(4, activation="softmax"))

#funcion para ir ajustando el learning rate durante el entrenamiento
def scheduler(epoch):
    if epoch < 30:
        return 0.0001
    else:
        return 0.0001 * tf.math.exp(0.1 * (10 - epoch))


reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.001)
#callback de la funcion para ajustar el learning rate
lr_schedule = tf.keras.callbacks.LearningRateScheduler(scheduler)
#callback para guardar los pesos de la epoch con mejor resultado de perdida en el set de validacion
checkpoint = ModelCheckpoint('covid_cnn_weights.h5', monitor='val_loss', verbose=1, save_best_only=True, mode = 'auto')
#callback de early stopping con paciencia hasta 8 epocas
stopping = EarlyStopping(monitor='val_loss', patience=8)
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [9]:
# import os, cv2
# img_folder = './balanced_dataset_medium/data/'
# n = 10000
# x_train = []
# y_train = []
# for file in dataInfo["ID"][:10000]:
#   image_path = os.path.join(img_folder, file+".jpg") #ruta de imagen
#   image = cv2.imread(image_path) #lectura de imagen
#   image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#   image = np.array(image) #se transforma a un numpy array para que sirva de input para la red
#   image = image.astype('float32')
#   image /= 255 #estandarización de imagen
#   x_train.append(image)
#   y_train.append(dataInfo.query("ID == @file")['label'].item())
#   #views = dataInfo.query("ID == @file")['views']

In [10]:
# for i in range(len(x_train)):
#   x_train[i] = np.array(x_train[i])

In [11]:
#x_train = np.array(x_train,dtype=np.uint16)
# x_train = np.array(x_train)

In [12]:
# x_axis = ["<10000", "<100k", "<1kk", ">1kk"]
# y_axis = [y_train.count(0),y_train.count(1),y_train.count(2),y_train.count(3)]
# print(y_axis[0],y_axis[1],y_axis[2],y_axis[3])
# plt.bar(x_axis, y_axis, width = 0.4)
# plt.title("YT-8M dataset balanced(?")
# plt.show()

In [13]:
# y_train = np.array(y_train)
# print(y_train)

In [14]:
#y_train = keras.utils.to_categorical(y_train, 4)

In [15]:
#plt.imshow(x_train[1])

In [18]:
# Entrenamos la red.
# El steps_per_epoch indica cuantos batches del train son un epoch.
# El validation_steps es lo mismo pero para el validation.
history = model.fit(trainGenerator, validation_data=valGenerator, epochs=100, callbacks = [stopping, lr_schedule, reduce_lr])

Epoch 1/100
374/374 [==============================] - 102s 259ms/step - loss: 1.3869 - accuracy: 0.2523 - val_loss: 1.3861 - val_accuracy: 0.2513
Epoch 2/100
374/374 [==============================] - 99s 264ms/step - loss: 1.3868 - accuracy: 0.2589 - val_loss: 1.3860 - val_accuracy: 0.2497
Epoch 3/100
301/374 [=======================>......] - ETA: 17s - loss: 1.3861 - accuracy: 0.2646

KeyboardInterrupt: ignored